# PubMed Papers: Get Abstracts
#### *Use pmidcite, requests, and BeautifulSoup to scrape posts from TowardsDataScience.com*

### *Query PubMed IDs using pmidcite and scrape NIH site for abstracts and citations

In [104]:
# Import libaries

import pandas as pd
import requests 
import numpy as np
from time import sleep 
from pmidcite.cfg import Cfg
from pmidcite.eutils.cmds.pubmed import PubMed

from bs4 import BeautifulSoup

In [105]:
cfg = Cfg(prt_fullname=False)

  READ: .pmidciterc


## Instantiate a PubMed object

In [106]:
pmobj = PubMed(
    email=cfg.get_email(),
    apikey=cfg.get_apikey(),
    tool=cfg.get_tool())

## Query PubMed, download PMIDs

In [107]:
pubmed_query = '("machine learning"[Title/Abstract]) AND ((fha[Filter]) AND (classicalarticle[Filter] OR journalarticle[Filter] OR meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter] OR technicalreport[Filter]) AND (2010/1/1:2020/12/31[pdat]) AND (english[Filter]))'
pmids = pmobj.dnld_query_pmids(pubmed_query)

 3,152 IDs FOR pubmed QUERY(("machine learning"[Title/Abstract]) AND ((fha[Filter]) AND (classicalarticle[Filter] OR journalarticle[Filter] OR meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter] OR technicalreport[Filter]) AND (2010/1/1:2020/12/31[pdat]) AND (english[Filter])))


In [108]:
pmid_df = pd.DataFrame(pmids, columns = ['pmid'])

In [118]:
pmid_df.to_csv('../capstone_project/data/list_of_pmids.csv',index=False)

### Scrape info from NIH Site 

In [110]:
#test url
url = 'https://pubmed.ncbi.nlm.nih.gov/31058383/'

#check status
res = requests.get(url)
res

<Response [200]>

In [111]:
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')

In [112]:
pmid_df = pd.read_csv('../data/list_of_pmids.csv')

In [113]:
abstract_list = []

batch = 1
counter = 1

for pmid in pmid_df['pmid']:
    abstract_dict = {}
    url = (f'https://pubmed.ncbi.nlm.nih.gov/{pmid}/')
    page  = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    #store pmid
    abstract_dict['pmid'] = pmid

    #get number of citations
    for row in soup.find_all('h2', {'class' : 'title'}):
        try: 
            abstract_dict['citations'] = int((row.find('em', {'class' : 'amount'}).text))
        except:
            pass

    #get abstract title
    try:
        abstract_dict['title'] = soup.find('h1').text.strip('\n \n \n \n \n \n \n')
    except:
        pass

    #get paper date
    try:
        abstract_dict['date'] = soup.find('span', {'class' : 'cit'}).text
    except:
        pass

    #get abstract text
    try:
        abstract_dict['text'] = soup.find('div', {'class' : 'abstract-content selected'}).text.replace('\n','').strip()
    except:
         pass
        
    counter +=1

    abstract_list.append(abstract_dict)
    sleep(np.random.randint(1, 5))
    
    if (counter % 100 == 0) or (counter == len(pmid_df['pmid'])):
        print('save here')
        df = pd.DataFrame(abstract_list)
        df.to_csv(f'data/raw_pubmed/{batch}_abstracts.csv',index=False)
        abstract_list = []
        batch +=1
    else: 
        pass

save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here
save here


In [114]:
df

,pmid,citations,title,date,text
0,21924741,29.0,Methods and tools for objective assessment of ...,2011 Nov;171(1):e81-95.,Training and assessment paradigms for laparosc...
1,21911058,27.0,Spelling with non-invasive Brain-Computer Inte...,Jan-Jun 2011;105(1-3):106-14.,Brain-Computer Interfaces (BCIs) have become a...
2,21864218,5.0,Computational modeling of P450s for toxicity p...,2011 Oct;7(10):1211-31.,Introduction: Drug developm...
3,21863503,1.0,"Representation, simulation, and hypothesis gen...",2011;759:465-82.,This chapter presents a discussion of metaboli...
4,21846786,161.0,Natural language processing: an introduction,Sep-Oct 2011;18(5):544-51.,Objectives: To provide an o...
5,21795890,22.0,Use of advanced machine-learning techniques fo...,2011 Jul;71(1 Suppl):S25-32.,Background: Hemorrhagic sho...
6,21787301,4.0,Machine learning algorithms for predicting pro...,2011 Sep;12(6):490-502.,Machine learning algorithms have wide range of...
7,21787299,8.0,Structural protein descriptors in 1-dimension ...,2011 Sep;12(6):470-89.,The last few decades observed an increasing in...
8,21787298,2.0,Cellular automata and its applications in prot...,2011 Sep;12(6):508-19.,With the explosion of protein sequences genera...
9,21780006,2.0,In silico prediction of post-translational mod...,2011;760:325-40.,Methods for predicting protein post-translatio...
